The CIFAR-10 dataset: download and pre-processing
-------------------------------------------------------

The CIFAR-10 dataset can be downloaded at [https://www.cs.toronto.edu/~kriz/](https://www.cs.toronto.edu/~kriz/) and it is a single compressed file containing serialized numpy arrays:

- [cifar-10-python.tar.gz](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz): test and training sets (163 MB)

The archive contains the files data_batch_1, data_batch_2, ..., data_batch_5, as well as test_batch. Each of these files is a Python "pickled" object produced with cPickle. The pickle files can be easilly deserialized using the following function:

In [ ]:
def unpickle(file):
    import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict


The unpickle() method reuturns a dictionary of elements. For instance the batches.meta file contains:

label_names: a 10 element list which gives meaningful names to the numeric labels in the labels array described above. For example, label_names[0] == "airplane", label_names[1] == "automobile", etc.

In [ ]:
my_dict = unpickle("./batches.meta")
labels_names = my_dict["label_names"]
print labels_names

The various *data_batch_x* files contain the following arrays:

**data**: numpy array (shape: 10000x3072) of uint8s. Each row of the array stores a 32x32 colour image. The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.

**labels**: list of 10000 numbers in the range 0-9. The number at index i indicates the label of the ith image in the array data.

In [ ]:
my_dict = unpickle("./data_batch_1")
for key, value in my_dict.items() :
    print (key)
    
training_set = my_dict[data]